In [1]:
# ===================================================================
# 車禍預測比賽 - PyTorch 多模態最終版 (更新參數)
# ===================================================================

# --- 0. 匯入必要套件 ---
import os
import pandas as pd
import numpy as np
import re
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from glob import glob
from tqdm import tqdm
import random
import gc

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix

# ===================================================================
#                        *** 1. 設定區 ***
# ===================================================================
class Config:
    # --- 1. 路徑、輸出與環境設定 ---
    BASE_IMG_PATH = '.'                   # 圖像資料夾 (road, freeway) 的根路徑
    FONT_PATH = 'Microsoft JhengHei'      # 用於 Matplotlib 顯示中文的字體
    RANDOM_STATE = 42                     # 固定隨機種子以確保實驗可重複
    
    # --- 2. 動態採樣策略設定 ---
    WINDOW_SIZE = 5                       # 每個採樣窗口要包含多少張圖片
    NUM_WINDOWS = 3                       # 總共要建立多少個採樣窗口 (組)
    
    # --- 3. 資料處理超參數 ---
    MAX_IMG_SEQ_LEN = WINDOW_SIZE         # 圖像序列長度現在由窗口大小決定
    MAX_TEXT_LEN = 512                    # VLM 文字序列的最大長度
    IMG_SIZE = (224, 224)                 # 圖像模型的標準輸入尺寸
    VOCAB_SIZE = 8000                     # 文字字典大小
    
    # --- 4. 模型架構超參數 ---
    # ✨ 可替換的模型建議: 'efficientnet_b1', 'resnet18', 'resnet34', 'mobilenetv3_large_100', 'efficientnet_b0',
    #                     'vit_base_patch16_clip_224.openai','swin_tiny_patch4_window7_224','convnext_tiny','maxvit_tiny_tf_224'
    MODEL_NAME = 'vit_base_patch16_clip_224.openai' # 使用的預訓練圖像模型
    EMBEDDING_DIM = 128                   # 文字嵌入層的維度
    TEXT_LSTM_UNITS = 64                  # 處理文字的 LSTM 神經元數量
    IMAGE_LSTM_UNITS = 64                 # 處理圖像序列的 LSTM 神經元數量
    DENSE_UNITS = 128                     # 融合後的全連接層神經元數量
    DROPOUT_RATE = 0.5                    # Dropout 層的比率
    
    # --- 5. 訓練過程超參數 ---
    USE_TWO_STAGE_FINETUNING = True       # 是否啟用兩階段微調策略?
    
    # --- 設定 (A): 兩階段微調適用 ---
    HEAD_EPOCHS = 120                     # [階段一] 只訓練頭部網路的最大回合數
    HEAD_LEARNING_RATE = 1e-4             # [階段一] 訓練頭部網路的學習率
    FINETUNE_EPOCHS = 120                 # [階段二] 解凍後進行微調的最大回合數
    FINETUNE_LEARNING_RATE = 1e-5         # [階段二] 微調時的學習率
    UNFREEZE_BLOCKS = -1                  # [階段二] 從圖像模型最後解凍幾個區塊 (-1 代表解凍全部)
    
    # --- 設定 (B): 單階段微調適用 ---
    SINGLE_STAGE_EPOCHS = 120             # [單階段] 全網路微調的最大回合數
    
    # --- 通用參數 ---
    LEARNING_RATE = 1e-4                  # [通用] 預設學習率 (在單階段模式下使用)
    WEIGHT_DECAY = 1e-3                   # AdamW 優化器的權重衰減
    LR_ETA_MIN = 1e-6                     # 學習率下降的最小值
    BATCH_SIZE = 8                        # 批次大小
    PATIENCE = 30                         # EarlyStopping 的耐心
    NUM_SPLITS = 5                        # 交叉驗證的摺數

    # --- 6. 進階策略開關 ---
    APPLY_MIXUP_CUTMIX = False            # 是否在訓練時啟用 MixUp/CutMix?
    MIXUP_CUTMIX_PROB = 0.5               # 應用 MixUp/CutMix 的機率
    MIXUP_ALPHA = 0.4                     # MixUp/CutMix 的混合強度參數
    APPLY_TTA = False                     # 是否在預測時啟用 TTA (測試時增強)?

# ===================================================================
#                  *** 2. 輔助工具與類別定義 ***
# ===================================================================

def set_environment(seed):
    torch.manual_seed(seed);
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True; torch.backends.cudnn.benchmark = False
    np.random.seed(seed); random.seed(seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\n[環境資訊] 隨機種子已設定為: {seed}"); print(f"[環境資訊] 執行設備: {device}")
    if torch.cuda.is_available(): print(f"[環境資訊] GPU 名稱: {torch.cuda.get_device_name(0)}")
    return device

def mixup_data(x, y, alpha=1.0, device='cuda'):
    if alpha > 0: lam = np.random.beta(alpha, alpha)
    else: lam = 1
    index = torch.randperm(x.size(0)).to(device); mixed_x = lam * x + (1 - lam) * x[index, :];
    return mixed_x, y, y[index], lam

def cutmix_data(x, y, alpha=1.0):
    if alpha > 0: lam = np.random.beta(alpha, alpha)
    else: lam = 1
    index = torch.randperm(x.size(0)).to(x.device); bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, :, bby1:bby2, bbx1:bbx2] = x[index, :, :, bby1:bby2, bbx1:bbx2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size()[-1] * x.size()[-2]))
    return x, y, y[index], lam

def rand_bbox(size, lam):
    W, H = size[-1], size[-2]; cut_rat = np.sqrt(1. - lam); cut_w, cut_h = int(W * cut_rat), int(H * cut_rat)
    cx, cy = np.random.randint(W), np.random.randint(H); bbx1, bby1 = np.clip(cx - cut_w // 2, 0, W), np.clip(cy - cut_h // 2, 0, H)
    bbx2, bby2 = np.clip(cx + cut_w // 2, 0, W), np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def mixup_criterion(criterion, pred, y_a, y_b, lam): return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

class SimpleTokenizer:
    def __init__(self, num_words=8000, oov_token="<OOV>"): self.word_index, self.num_words, self.oov_token = {oov_token: 1}, num_words, oov_token
    def fit_on_texts(self, texts):
        word_counts = pd.Series([word for text in texts for word in text.split()]).value_counts()
        for i, (word, _) in enumerate(word_counts.head(self.num_words - 2).items()): self.word_index[word] = i + 2
    def texts_to_sequences(self, texts): return [[self.word_index.get(word, 1) for word in text.split()] for text in texts]

class DashcamDataset(Dataset):
    def __init__(self, df, tokenizer, config, transform, sampling_strategy, is_test=False):
        self.df, self.tokenizer, self.config, self.transform, self.sampling_strategy, self.is_test = df, tokenizer, config, transform, sampling_strategy, is_test
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]; text_seq = self.tokenizer.texts_to_sequences([row['processed_text']])[0]
        padded_text = np.zeros(self.config.MAX_TEXT_LEN, dtype=np.int64)
        if len(text_seq) > 0:
            if len(text_seq) > self.config.MAX_TEXT_LEN: text_seq = text_seq[-self.config.MAX_TEXT_LEN:]
            padded_text[-len(text_seq):] = text_seq
        image_sequence = torch.zeros(self.config.MAX_IMG_SEQ_LEN, 3, *self.config.IMG_SIZE)
        all_paths = row['image_paths']
        start_idx, end_idx = self.sampling_strategy['start'], self.sampling_strategy['end']
        paths_to_load = all_paths[start_idx:end_idx] if len(all_paths) >= abs(start_idx) else []
        for j, img_path in enumerate(paths_to_load):
            if j >= self.config.MAX_IMG_SEQ_LEN: break
            try: img = Image.open(img_path).convert('RGB'); image_sequence[j] = self.transform(img)
            except Exception: continue
        if self.is_test: return torch.tensor(padded_text, dtype=torch.long), image_sequence, row['file_name']
        else: return torch.tensor(padded_text, dtype=torch.long), image_sequence, torch.tensor(row['risk'], dtype=torch.long)

class MultimodalModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config; self.embedding = nn.Embedding(config.VOCAB_SIZE, config.EMBEDDING_DIM, padding_idx=0)
        self.text_lstm = nn.LSTM(config.EMBEDDING_DIM, config.TEXT_LSTM_UNITS, batch_first=True, bidirectional=True); self.text_dropout = nn.Dropout(config.DROPOUT_RATE)
        self.image_encoder = timm.create_model(config.MODEL_NAME, pretrained=True, num_classes=0, global_pool='avg')
        self.image_lstm = nn.LSTM(self.image_encoder.num_features, config.IMAGE_LSTM_UNITS, batch_first=True, bidirectional=True); self.image_dropout = nn.Dropout(config.DROPOUT_RATE)
        fusion_dim = (config.TEXT_LSTM_UNITS * 2) + (config.IMAGE_LSTM_UNITS * 2)
        self.classifier = nn.Sequential(nn.Linear(fusion_dim, config.DENSE_UNITS), nn.ReLU(), nn.Dropout(config.DROPOUT_RATE), nn.Linear(config.DENSE_UNITS, 2))
        print(f"PyTorch 多模態模型 ({config.MODEL_NAME}) 建立完成。")
    def freeze_backbone(self):
        print("--- [策略] 凍結圖像主幹網路權重 ---")
        for param in self.image_encoder.parameters(): param.requires_grad = False
    def unfreeze_backbone(self, unfreeze_blocks=-1):
        if unfreeze_blocks == -1:
            print("--- [策略] 解凍整個圖像主幹網路 ---")
            for param in self.image_encoder.parameters(): param.requires_grad = True
        elif unfreeze_blocks > 0 and hasattr(self.image_encoder, 'blocks'):
            print(f"--- [策略] 解凍圖像主幹網路的最後 {unfreeze_blocks} 個區塊 ---")
            total_blocks = len(self.image_encoder.blocks)
            for i in range(total_blocks - unfreeze_blocks, total_blocks):
                for param in self.image_encoder.blocks[i].parameters(): param.requires_grad = True
            for param in getattr(self.image_encoder, 'conv_head', []).parameters(): param.requires_grad = True
            for param in getattr(self.image_encoder, 'bn2', []).parameters(): param.requires_grad = True
    def forward(self, text, image):
        text_embeds = self.embedding(text); _, (text_hidden, _) = self.text_lstm(text_embeds); text_features = torch.cat((text_hidden[-2,:,:], text_hidden[-1,:,:]), dim=1); text_features = self.text_dropout(text_features)
        b, t, c, h, w = image.shape; img_flat = image.view(b * t, c, h, w); img_features = self.image_encoder(img_flat); img_features = img_features.view(b, t, -1); _, (img_hidden, _) = self.image_lstm(img_features); img_features = torch.cat((img_hidden[-2,:,:], img_hidden[-1,:,:]), dim=1); img_features = self.image_dropout(img_features)
        return self.classifier(torch.cat((text_features, img_features), dim=1))

# ===================================================================
#                     *** 3. 主執行流程 ***
# ===================================================================
def main():
    config = Config()
    device = set_environment(config.RANDOM_STATE)
    config.SAMPLING_STRATEGIES = []
    for i in range(1, config.NUM_WINDOWS + 1):
        start_offset = (i - 1) * config.WINDOW_SIZE + 1; end_offset = i * config.WINDOW_SIZE
        config.SAMPLING_STRATEGIES.append({'name': f'Last_{start_offset}_to_{end_offset}', 'start': -end_offset, 'end': None if i == 1 else -(end_offset - config.WINDOW_SIZE)})
    print("\n" + "="*15, "動態生成的採樣策略", "="*15); [print(s) for s in config.SAMPLING_STRATEGIES]; print("-" * 50)
    TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
    tech_parts = [];
    if config.APPLY_MIXUP_CUTMIX: tech_parts.append("MixCut")
    if config.APPLY_TTA: tech_parts.append("TTA")
    tech_str = "+".join(tech_parts) if tech_parts else "Base"
    ft_str = "2StageFT" if config.USE_TWO_STAGE_FINETUNING else "1StageFT"
    EXPERIMENT_NAME = f"{config.MODEL_NAME}_{ft_str}_{tech_str}_{config.MAX_IMG_SEQ_LEN}F_{config.BATCH_SIZE}B_{TIMESTAMP}"
    MAIN_OUTPUT_DIR = os.path.join('.', f'output_pytorch_MultiWindow_{TIMESTAMP}')
    os.makedirs(MAIN_OUTPUT_DIR, exist_ok=True)
    print(f"所有策略的輸出將儲存至主資料夾: {MAIN_OUTPUT_DIR}")
    try: plt.rcParams['font.sans-serif'] = [config.FONT_PATH]; plt.rcParams['axes.unicode_minus'] = False
    except Exception as e: print(f"[警告] 設定中文字體失敗: {e}。")

    print("\n" + "="*15, "步驟 A: 準備資料框架", "="*15)
    def get_paths_and_counts(df, base_path_map):
        img_paths_list = [sorted(glob(os.path.join(base_path_map['road' if row['file_name'].startswith('road') else 'freeway'], row['file_name'], '*.jpg'))) or sorted(glob(os.path.join(base_path_map['road' if row['file_name'].startswith('road') else 'freeway'], row['file_name'], '*.JPG'))) for _, row in df.iterrows()]
        df['image_paths'] = img_paths_list; df['image_count'] = [len(p) for p in img_paths_list]
        return df
    train_df = pd.concat([pd.read_csv('road_train_with_captions.csv'), pd.read_csv('freeway_train_with_captions.csv')], ignore_index=True)
    train_base_path_map = {'road': os.path.join(config.BASE_IMG_PATH, 'road', 'train'), 'freeway': os.path.join(config.BASE_IMG_PATH, 'freeway', 'train')}
    train_df = get_paths_and_counts(train_df, train_base_path_map); train_df = train_df[train_df['image_count'] > 0].reset_index(drop=True)
    test_captions_df = pd.concat([pd.read_csv('road_test_with_captions.csv'), pd.read_csv('freeway_test_with_captions.csv')], ignore_index=True)
    sub_df = pd.read_csv('sample_submission.csv')
    test_df = pd.merge(sub_df[['file_name']], test_captions_df, on='file_name', how='left')
    test_base_path_map = {'road': os.path.join(config.BASE_IMG_PATH, 'road', 'test'), 'freeway': os.path.join(config.BASE_IMG_PATH, 'freeway', 'test')}
    test_df = get_paths_and_counts(test_df, test_base_path_map)
    def preprocess_text(text): return ' '.join([re.sub(r'[^a-z0-9\s]', '', s.lower()).strip() for s in str(text).split('|')])
    train_df['processed_text'] = train_df['captions'].apply(preprocess_text)
    if not test_df.empty: test_df['processed_text'] = test_df['captions'].apply(preprocess_text)
    tokenizer = SimpleTokenizer(num_words=config.VOCAB_SIZE); tokenizer.fit_on_texts(train_df['processed_text'])
    joblib.dump(tokenizer, os.path.join(MAIN_OUTPUT_DIR, 'tokenizer.pkl'))
    print("資料預處理與 Tokenizer 建立完成。"); print("-" * 50)
    transform = transforms.Compose([transforms.Resize(config.IMG_SIZE), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    grand_ensemble_models, grand_ensemble_test_probs = [], []
    for strategy in config.SAMPLING_STRATEGIES:
        strategy_name = strategy['name']; print(f"\n" + "#"*25, f"開始處理策略: {strategy_name}", "#"*25)
        STRATEGY_OUTPUT_DIR = os.path.join(MAIN_OUTPUT_DIR, strategy_name); os.makedirs(STRATEGY_OUTPUT_DIR, exist_ok=True)
        best_models_info_per_strategy = []
        skf = StratifiedKFold(n_splits=config.NUM_SPLITS, shuffle=True, random_state=config.RANDOM_STATE)
        for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['risk'])):
            print(f"\n" + "="*20, f"Fold {fold + 1}/{config.NUM_SPLITS} for {strategy_name}", "="*20)
            train_fold_df, val_fold_df = train_df.iloc[train_idx], train_df.iloc[val_idx]
            train_dataset = DashcamDataset(train_fold_df, tokenizer, config, transform, sampling_strategy=strategy); val_dataset = DashcamDataset(val_fold_df, tokenizer, config, transform, sampling_strategy=strategy)
            train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=0); val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0)
            model = MultimodalModel(config).to(device)
            class_counts = train_fold_df['risk'].value_counts().sort_index()
            weights = 1.0 / torch.tensor(class_counts.values, dtype=torch.float32) if len(class_counts) > 1 else None
            if weights is not None: weights = weights / weights.sum()
            criterion = nn.CrossEntropyLoss(weight=weights.to(device) if weights is not None else None)
            
            if config.USE_TWO_STAGE_FINETUNING:
                print("\n--- STAGE 1: 開始訓練分類器頭部 ---"); model.freeze_backbone()
                optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=config.HEAD_LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.HEAD_EPOCHS, eta_min=config.LR_ETA_MIN)
                best_score, patience_counter = -1, 0; model_path = os.path.join(STRATEGY_OUTPUT_DIR, f'temp_best_head_fold_{fold+1}.pth')
                for epoch in range(config.HEAD_EPOCHS):
                    print(f"\n--- [S1] Epoch {epoch + 1}/{config.HEAD_EPOCHS} ---"); model.train()
                    for text, image, labels in tqdm(train_loader, desc=f"[S1] F{fold+1} {strategy_name}"):
                        text, image, labels = text.to(device), image.to(device), labels.to(device)
                        if config.APPLY_MIXUP_CUTMIX and np.random.rand() < config.MIXUP_CUTMIX_PROB:
                            mixed_image, y_a, y_b, lam = mixup_data(image, labels, config.MIXUP_ALPHA, device) if np.random.rand() > 0.5 else cutmix_data(image, labels, config.MIXUP_ALPHA)
                            outputs = model(text, mixed_image); loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
                        else: outputs = model(text, image); loss = criterion(outputs, labels)
                        optimizer.zero_grad(); loss.backward(); optimizer.step()
                    scheduler.step(); model.eval(); all_probs, all_labels = [], []
                    with torch.no_grad():
                        for text, image, labels in tqdm(val_loader, desc="[S1] 驗證中"): outputs = model(text.to(device), image.to(device)); all_probs.extend(torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()); all_labels.extend(labels.cpu().numpy())
                    val_probs, val_labels = np.array(all_probs), np.array(all_labels)
                    accuracy = accuracy_score(val_labels, val_probs > 0.5); f1 = f1_score(val_labels, val_probs > 0.5); 
                    try: auc = roc_auc_score(val_labels, val_probs)
                    except ValueError: auc = 0.5
                    score = f1 * 0.5 + accuracy * 0.3 + auc * 0.2
                    print(f"[S1] Epoch {epoch + 1} 結果: Val Score={score:.4f} (F1:{f1:.4f}, Acc:{accuracy:.4f}, AUC:{auc:.4f})")
                    if score > best_score: best_score = score; patience_counter = 0; torch.save(model.state_dict(), model_path); print(f"✨ [S1] 新的最佳分數！暫存頭部模型。")
                    else: patience_counter += 1
                    if patience_counter >= config.PATIENCE: print("[S1] 觸發早停！"); break
                
                print("\n--- STAGE 2: 開始微調整個模型 ---"); model.load_state_dict(torch.load(model_path)); model.unfreeze_backbone(config.UNFREEZE_BLOCKS)
                optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=config.FINETUNE_LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.FINETUNE_EPOCHS, eta_min=config.LR_ETA_MIN)
                patience_counter = 0; best_finetune_score = best_score
                for epoch in range(config.FINETUNE_EPOCHS):
                    print(f"\n--- [S2] Epoch {epoch + 1}/{config.FINETUNE_EPOCHS} ---"); model.train()
                    for text, image, labels in tqdm(train_loader, desc=f"[S2] F{fold+1} {strategy_name}"):
                        text, image, labels = text.to(device), image.to(device), labels.to(device)
                        optimizer.zero_grad(); outputs = model(text, image); loss = criterion(outputs, labels); loss.backward(); optimizer.step()
                    scheduler.step(); model.eval(); all_probs, all_labels = [], [];
                    with torch.no_grad():
                        for text, image, labels in tqdm(val_loader, desc="[S2] 驗證中"):
                             outputs = model(text.to(device), image.to(device)); all_probs.extend(torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()); all_labels.extend(labels.cpu().numpy())
                    val_probs, val_labels = np.array(all_probs), np.array(all_labels)
                    accuracy = accuracy_score(val_labels, val_probs > 0.5); f1 = f1_score(val_labels, val_probs > 0.5);
                    try: auc = roc_auc_score(val_labels, val_probs)
                    except ValueError: auc = 0.5
                    score = f1 * 0.5 + accuracy * 0.3 + auc * 0.2
                    print(f"[S2] Epoch {epoch + 1} 結果: Val Score={score:.4f} (F1:{f1:.4f}, Acc:{accuracy:.4f}, AUC:{auc:.4f})")
                    if score > best_finetune_score:
                        best_finetune_score = score; patience_counter = 0; torch.save(model.state_dict(), model_path); print(f"✨ [S2] 新的最佳分數！模型已儲存。")
                    else: patience_counter += 1
                    if patience_counter >= config.PATIENCE: print("[S2] 觸發早停！"); break
                best_models_info_per_strategy.append({'path': model_path, 'score': best_finetune_score})
            else: # 單階段全網路微調
                print("\n--- [單階段策略] 開始全網路微調 ---"); model.unfreeze_backbone(unfreeze_blocks=-1)
                optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.SINGLE_STAGE_EPOCHS, eta_min=config.LR_ETA_MIN)
                best_score, patience_counter = -1, 0; model_path = os.path.join(STRATEGY_OUTPUT_DIR, f'best_model_fold_{fold+1}.pth')
                for epoch in range(config.SINGLE_STAGE_EPOCHS):
                    print(f"\n--- [單階段] Epoch {epoch + 1}/{config.SINGLE_STAGE_EPOCHS} ---"); model.train()
                    for text, image, labels in tqdm(train_loader, desc=f"[單階段][F{fold+1}] {strategy_name}"):
                        text, image, labels = text.to(device), image.to(device), labels.to(device)
                        if config.APPLY_MIXUP_CUTMIX and np.random.rand() < config.MIXUP_CUTMIX_PROB:
                            mixed_image, y_a, y_b, lam = mixup_data(image, labels, config.MIXUP_ALPHA, device) if np.random.rand() > 0.5 else cutmix_data(image, labels, config.MIXUP_ALPHA)
                            outputs = model(text, mixed_image); loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
                        else: outputs = model(text, image); loss = criterion(outputs, labels)
                        optimizer.zero_grad(); loss.backward(); optimizer.step()
                    scheduler.step(); model.eval(); all_probs, all_labels = [], [];
                    with torch.no_grad():
                        for text, image, labels in tqdm(val_loader, desc=f"[單階段][F{fold+1}] {strategy_name}"):
                            outputs = model(text.to(device), image.to(device)); all_probs.extend(torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()); all_labels.extend(labels.cpu().numpy())
                    val_probs, val_labels = np.array(all_probs), np.array(all_labels)
                    accuracy = accuracy_score(val_labels, val_probs > 0.5); f1 = f1_score(val_labels, val_probs > 0.5);
                    try: auc = roc_auc_score(val_labels, val_probs)
                    except ValueError: auc = 0.5
                    score = f1 * 0.5 + accuracy * 0.3 + auc * 0.2
                    print(f"[單階段] Epoch {epoch + 1} 結果: Val Score={score:.4f} (F1:{f1:.4f}, Acc:{accuracy:.4f}, AUC:{auc:.4f})")
                    if score > best_score:
                        best_score = score; patience_counter = 0; torch.save(model.state_dict(), model_path); print(f"✨ [單階段] 新的最佳分數！模型已儲存。")
                    else: patience_counter += 1
                    if patience_counter >= config.PATIENCE: print("[單階段] 觸發早停！"); break
                best_models_info_per_strategy.append({'path': model_path, 'score': best_score})
            grand_ensemble_models.append(best_models_info_per_strategy[-1])
            del model, train_loader, val_loader; gc.collect(); torch.cuda.empty_cache()
        if not test_df.empty and best_models_info_per_strategy:
            print(f"\n--- 對策略 [{strategy_name}] 的結果進行集成預測 ---")
            test_dataset = DashcamDataset(test_df, tokenizer, config, transform, sampling_strategy=strategy, is_test=True)
            test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0)
            strategy_fold_probs = []
            for model_info in best_models_info_per_strategy:
                if not os.path.exists(model_info['path']): continue
                model = MultimodalModel(config).to(device); model.load_state_dict(torch.load(model_info['path'])); model.eval(); fold_probs = []
                with torch.no_grad():
                    for text, image, _ in tqdm(test_loader, desc=f"預測中 (Fold Model Score: {model_info['score']:.4f})"):
                        outputs = model(text.to(device), image.to(device)); fold_probs.extend(torch.softmax(outputs, dim=1)[:, 1].cpu().numpy())
                strategy_fold_probs.append(fold_probs)
            if strategy_fold_probs:
                strategy_final_probs = np.mean(strategy_fold_probs, axis=0); grand_ensemble_test_probs.append(strategy_final_probs)
                strategy_predictions = (strategy_final_probs > 0.5).astype(int)
                submission_df = pd.DataFrame({'file_name': test_df['file_name'].tolist(), 'risk': strategy_predictions})
                submission_path = os.path.join(STRATEGY_OUTPUT_DIR, f'submission_{strategy_name}.csv')
                submission_df.to_csv(submission_path, index=False)
                print(f"✅ 策略 [{strategy_name}] 的預測結果已儲存至: {submission_path}")
    if grand_ensemble_models and not test_df.empty and len(grand_ensemble_test_probs) > 1:
        print("\n" + "="*25, "開始終極集成預測", "="*25)
        grand_final_probs = np.mean(grand_ensemble_test_probs, axis=0); grand_final_predictions = (grand_final_probs > 0.5).astype(int)
        submission_df = pd.DataFrame({'file_name': test_df['file_name'].tolist(), 'risk': grand_final_predictions})
        submission_filename = f'submission_GRAND_ENSEMBLE_{TIMESTAMP}.csv'
        submission_path = os.path.join(MAIN_OUTPUT_DIR, submission_filename)
        submission_df.to_csv(submission_path, index=False)
        print(f"\n✅✅✅ 終極集成預測結果已儲存至: {submission_path}"); print("最終預測分佈:"); print(submission_df['risk'].value_counts())
        print("\n" + "="*20, "所有策略與 Fold 最終總結", "="*20)
        all_scores = [info['score'] for info in grand_ensemble_models]
        avg_score = np.mean(all_scores); std_score = np.std(all_scores)
        print(f"📊 總共 {len(all_scores)} 個模型的平均驗證分數: {avg_score:.4f} ± {std_score:.4f}")
        summary_df = pd.DataFrame(grand_ensemble_models); summary_path = os.path.join(MAIN_OUTPUT_DIR, f"training_summary_GRAND_ENSEMBLE_{TIMESTAMP}.csv")
        summary_df.to_csv(summary_path, index=False, float_format='%.4f'); 
        print(f"\n✅ 完整訓練分數總結已儲存至: {summary_path}")

    print("\n🎉🎉🎉 所有流程執行完畢！🎉🎉🎉")

if __name__ == '__main__':
    main()


[環境資訊] 隨機種子已設定為: 42
[環境資訊] 執行設備: cuda
[環境資訊] GPU 名稱: NVIDIA GeForce RTX 3060 Ti

=============== 動態生成的採樣策略 ===============
{'name': 'Last_1_to_5', 'start': -5, 'end': None}
{'name': 'Last_6_to_10', 'start': -10, 'end': -5}
{'name': 'Last_11_to_15', 'start': -15, 'end': -10}
--------------------------------------------------
所有策略的輸出將儲存至主資料夾: .\output_pytorch_MultiWindow_20250710_131200

=============== 步驟 A: 準備資料框架 ===============
資料預處理與 Tokenizer 建立完成。
--------------------------------------------------

######################### 開始處理策略: Last_1_to_5 #########################

==================== Fold 1/5 for Last_1_to_5 ====================


Unexpected keys (norm.weight, norm.bias) found while loading pretrained weights. This may be expected if model is being adapted.


PyTorch 多模態模型 (vit_base_patch16_clip_224.openai) 建立完成。

--- STAGE 1: 開始訓練分類器頭部 ---
--- [策略] 凍結圖像主幹網路權重 ---

--- [S1] Epoch 1/120 ---


[S1] 驗證中: 100%|███████████████████████████████████████████████████████████████████████| 9/9 [00:04<00:00,  1.97it/s]


[S1] Epoch 1 結果: Val Score=0.3368 (F1:0.0667, Acc:0.6111, AUC:0.6007)
✨ [S1] 新的最佳分數！暫存頭部模型。

--- [S1] Epoch 2/120 ---


...

PyTorch 多模態模型 (vit_base_patch16_clip_224.openai) 建立完成。


預測中 (Fold Model Score: 0.6516): 100%|███████████████████████████████████████████████| 31/31 [00:14<00:00,  2.12it/s]
Unexpected keys (norm.weight, norm.bias) found while loading pretrained weights. This may be expected if model is being adapted.


PyTorch 多模態模型 (vit_base_patch16_clip_224.openai) 建立完成。


預測中 (Fold Model Score: 0.5871): 100%|███████████████████████████████████████████████| 31/31 [00:14<00:00,  2.14it/s]

✅ 策略 [Last_11_to_15] 的預測結果已儲存至: .\output_pytorch_MultiWindow_20250710_131200\Last_11_to_15\submission_Last_11_to_15.csv

========================= 開始終極集成預測 =========================

✅✅✅ 終極集成預測結果已儲存至: .\output_pytorch_MultiWindow_20250710_131200\submission_GRAND_ENSEMBLE_20250710_131200.csv
最終預測分佈:
risk
0    150
1     93
Name: count, dtype: int64

==================== 所有策略與 Fold 最終總結 ====================
📊 總共 15 個模型的平均驗證分數: 0.6199 ± 0.0417

✅ 完整訓練分數總結已儲存至: .\output_pytorch_MultiWindow_20250710_131200\training_summary_GRAND_ENSEMBLE_20250710_131200.csv

🎉🎉🎉 所有流程執行完畢！🎉🎉🎉
